In [1]:
import requests
from pymongo import MongoClient
import xmltodict
data_path_post = "../data/mongodb_xml/ai.stackexchange.com/Posts.xml"
data_path_user = "../data/mongodb_xml/ai.stackexchange.com/Users.xml"

In [2]:
client = MongoClient('mongodb',27017)
db = client['mabdd']
Posts = db['Posts']
Posts.delete_many({})
Users = db['Users']
Users.delete_many({})


In [3]:
with open(data_path_post,'r') as file :
    line = file.readline()
    print(line)
    line = file.readline()
    print(line)
    line = file.readline()
    print(line)
    dict_test = xmltodict.parse(line)
    print(dict_test)
    print("Titre : \n",dict_test['row']['@Title'])

﻿<?xml version="1.0" encoding="utf-8"?>

<posts>

  <row Id="1" PostTypeId="1" AcceptedAnswerId="3" CreationDate="2016-08-02T15:39:14.947" Score="8" ViewCount="384" Body="&lt;p&gt;What does &quot;backprop&quot; mean? I've Googled it, but it's showing backpropagation.&lt;/p&gt;&#xA;&#xA;&lt;p&gt;Is the &quot;backprop&quot; term basically the same as &quot;backpropagation&quot; or does it have a different meaning?&lt;/p&gt;&#xA;" OwnerUserId="8" LastEditorUserId="10135" LastEditDate="2018-10-18T10:45:18.660" LastActivityDate="2018-10-18T10:45:18.660" Title="What is &quot;backprop&quot;?" Tags="&lt;neural-networks&gt;&lt;backpropagation&gt;&lt;terminology&gt;&lt;definitions&gt;" AnswerCount="3" CommentCount="3" FavoriteCount="1" />

OrderedDict([('row', OrderedDict([('@Id', '1'), ('@PostTypeId', '1'), ('@AcceptedAnswerId', '3'), ('@CreationDate', '2016-08-02T15:39:14.947'), ('@Score', '8'), ('@ViewCount', '384'), ('@Body', '<p>What does "backprop" mean? I\'ve Googled it, but it\'s showing

## Q1

In [4]:
def insert_xml_mongodb(collection, xml_path):
    with open(xml_path,'r') as file :
        for line in file.readlines():
            try :
                dict_post = xmltodict.parse(line)
                collection.insert_one(dict_post["row"])
            except :
                print(line)

In [5]:
Posts.find_one({})

In [6]:
insert_xml_mongodb(Posts, data_path_post)

﻿<?xml version="1.0" encoding="utf-8"?>

<posts>

</posts>


In [7]:
Posts.find_one({})

{'_id': ObjectId('5d0cbd7fe418eb06623b98e2'),
 '@Id': '1',
 '@PostTypeId': '1',
 '@AcceptedAnswerId': '3',
 '@CreationDate': '2016-08-02T15:39:14.947',
 '@Score': '8',
 '@ViewCount': '384',
 '@Body': '<p>What does "backprop" mean? I\'ve Googled it, but it\'s showing backpropagation.</p>\n\n<p>Is the "backprop" term basically the same as "backpropagation" or does it have a different meaning?</p>\n',
 '@OwnerUserId': '8',
 '@LastEditorUserId': '10135',
 '@LastEditDate': '2018-10-18T10:45:18.660',
 '@LastActivityDate': '2018-10-18T10:45:18.660',
 '@Title': 'What is "backprop"?',
 '@Tags': '<neural-networks><backpropagation><terminology><definitions>',
 '@AnswerCount': '3',
 '@CommentCount': '3',
 '@FavoriteCount': '1'}

In [8]:
insert_xml_mongodb(Users, data_path_user)

﻿<?xml version="1.0" encoding="utf-8"?>

<users>

</users>


# Q2

In [9]:
cursor = Users.aggregate([
    {"$project":{"arrayofkeyvalue":{"$objectToArray":"$$ROOT"}}},
    {"$unwind":"$arrayofkeyvalue"},
    {"$group":{"_id":None,"allkeys":{"$addToSet":"$arrayofkeyvalue.k"}}} ])
[r for r in cursor]

[{'_id': None,
  'allkeys': ['@WebsiteUrl',
   '@AccountId',
   '@DownVotes',
   '@Reputation',
   '@Id',
   '@DisplayName',
   '@UpVotes',
   '_id',
   '@LastAccessDate',
   '@ProfileImageUrl',
   '@Location',
   '@Views',
   '@CreationDate',
   '@AboutMe']}]

In [10]:
cursor = Posts.aggregate([
    {"$project":{"arrayofkeyvalue":{"$objectToArray":"$$ROOT"}}},
    {"$unwind":"$arrayofkeyvalue"},
    {"$group":{"_id":None,"allkeys":{"$addToSet":"$arrayofkeyvalue.k"}}} ])
cursor.next()["allkeys"]

['@LastEditorDisplayName',
 '@ClosedDate',
 '@ParentId',
 '@OwnerDisplayName',
 '@FavoriteCount',
 '@CommentCount',
 '@AnswerCount',
 '@LastActivityDate',
 '@LastEditDate',
 '@AcceptedAnswerId',
 '@ViewCount',
 '@PostTypeId',
 '@Id',
 '@Tags',
 '@Title',
 '_id',
 '@LastEditorUserId',
 '@CreationDate',
 '@CommunityOwnedDate',
 '@Score',
 '@OwnerUserId',
 '@Body']

# Q3

In [11]:
Posts.distinct("@PostTypeId")

['1', '2', '5', '4', '7']

# Q4

In [12]:
def insert_xml_Post2(collection, xml_path):
    with open(xml_path,'r') as file :
        for line in file.readlines():
            try :
                dict_post = xmltodict.parse(line)["row"]
                if dict_post["@PostTypeId"]=='2' and "@ParentId" in dict_post.keys():
                    # print("a")
                    #print(collection.find_one({'@Id':dict_post["@ParentId"]}))
                    collection.update_one({'@Id':dict_post["@ParentId"]},
                                      {"$push":{"Response": dict_post}},
                                      upsert=True)
                elif dict_post["@PostTypeId"]=='1' :
                    collection.insert_one(dict_post)
            except :
                
                print(line)

In [13]:
Posts2 = db['Posts2']
Posts2.delete_many({})

In [14]:
insert_xml_Post2(Posts2, data_path_post)

﻿<?xml version="1.0" encoding="utf-8"?>

<posts>

</posts>


In [15]:
Posts2.find_one({"Response":{"$exists":True}})

{'_id': ObjectId('5d0cbda0e418eb06623c1753'),
 '@Id': '1',
 '@PostTypeId': '1',
 '@AcceptedAnswerId': '3',
 '@CreationDate': '2016-08-02T15:39:14.947',
 '@Score': '8',
 '@ViewCount': '384',
 '@Body': '<p>What does "backprop" mean? I\'ve Googled it, but it\'s showing backpropagation.</p>\n\n<p>Is the "backprop" term basically the same as "backpropagation" or does it have a different meaning?</p>\n',
 '@OwnerUserId': '8',
 '@LastEditorUserId': '10135',
 '@LastEditDate': '2018-10-18T10:45:18.660',
 '@LastActivityDate': '2018-10-18T10:45:18.660',
 '@Title': 'What is "backprop"?',
 '@Tags': '<neural-networks><backpropagation><terminology><definitions>',
 '@AnswerCount': '3',
 '@CommentCount': '3',
 '@FavoriteCount': '1',
 'Response': [{'@Id': '3',
   '@PostTypeId': '2',
   '@ParentId': '1',
   '@CreationDate': '2016-08-02T15:40:24.820',
   '@Score': '12',
   '@Body': '<p>"Backprop" is the same as "backpropagation": it\'s just a shorter way to say it. It is sometimes abbreviated as "BP".</p>

In [16]:
cursor = Users.find().sort("@Reputation")
l = [r for r in cursor]

In [17]:
l

[{'_id': ObjectId('5d0cbd87e418eb06623bbec9'),
  '@Id': '-1',
  '@Reputation': '1',
  '@CreationDate': '2016-08-02T00:14:10.580',
  '@DisplayName': 'Community',
  '@LastAccessDate': '2016-08-02T00:14:10.580',
  '@Location': 'on the server farm',
  '@AboutMe': '<p>Hi, I\'m not really a person.</p>\r\n<p>I\'m a background process that helps keep this site clean!</p>\r\n<p>I do things like</p>\r\n<ul>\r\n<li>Randomly poke old unanswered questions every hour so they get some attention</li>\r\n<li>Own community questions and answers so nobody gets unnecessary reputation from them</li>\r\n<li>Own downvotes on spam/evil posts that get permanently deleted</li>\r\n<li>Own suggested edits from anonymous users</li>\r\n<li><a href="http://meta.stackoverflow.com/a/92006">Remove abandoned questions</a></li>\r\n</ul>',
  '@Views': '0',
  '@UpVotes': '3',
  '@DownVotes': '661',
  '@AccountId': '-1'},
 {'_id': ObjectId('5d0cbd87e418eb06623bbed0'),
  '@Id': '7',
  '@Reputation': '1',
  '@CreationDate': 

In [ ]:
pou9ch